In [4]:
import numpy as np
from numpy import random as rnd

In [5]:
# Use dispersity (between 1 and 3) to choose which system to return a radius from

def pickRadius(dispersity):
    if (dispersity == 1):
        return (1/((np.pi)**0.5))
    if (dispersity == 2):
        a = np.array([((3/(2*(np.pi)))**0.5), ((1/(2*(np.pi)))**0.5)])
        return rnd.choice(a , p = [0.5, 0.5])
    if (dispersity == 3):
        a = np.array([((15/(16*(np.pi)))**0.5), ((5/(4*(np.pi)))**0.5)])
        return rnd.choice(a , p = [0.8, 0.2])

In [6]:
#Class for creating a disc
# Cooridinates represent middle of discs
# choose 1-3 for dispersity

class config(): 
    def __init__(self, dispersity):
        self.radius = pickRadius(dispersity)
        self.x = (rnd.rand()) * 20
        self.y = (rnd.rand()) * 20
        return

In [109]:
# iterates attempts of N
#select dispersity scenario of 1-3

def addDiscs(N, dispersity, overlapConstant = 1.0):
    # situ = np.empty(N, dtype = "object")
    situ = np.array([])
    
    for i in range(N):
        a = config(dispersity)
        #check if disc is hitting side
        insideBox = (
            ((a.x - a.radius > 0) and (a.x + a.radius < 20)) 
            and ((a.y - a.radius > 0) and (a.y + a.radius < 20))
        )
        print("inside box", insideBox)
        
        #checking hitting any other disc, accoutning for if its first
        # Need to check if length of array >0 first
        if (situ.size == 0):
            notHitting = True
        else:
            notHitting = np.all(
                ((((a.x - situ.x)**2) + ((a.y - situ.y)**2))**0.5) >
                (overlapConstant * (a.radius + situ.radius)))
        print("not hitting", notHitting)
        
        # appends disc to array if it meets conditions
        if (notHitting and insideBox):
            print("CONDITIONS MET")
            situ = np.append(situ, a)
            
        # Returns completed array after end of loop
        
    return situ
        

In [138]:
testArray = addDiscs(10,1)

print(testArray)
print(type(testArray))
# issue is that it's checking with empty items, maybe consider popping
# popping would change index number, skipping the following value

#could change i if one is removed
# masking would still have empty values... I think... we would want it to refrence the previous obj

# needs to check for EVERY PREVIOUS DISC

inside box False
not hitting True
inside box True
not hitting True
CONDITIONS MET
inside box True


AttributeError: 'numpy.ndarray' object has no attribute 'x'

In [114]:
test = np.empty(0, dtype = "void")



print(test.size)


0
